In [0]:
GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/rbf/phoenix/phoenix_HiRes_GK/pca'
#GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/rbf/bosz/bosz_50000_GK/pca'

In [0]:
%matplotlib inline

In [0]:
import sys

# Allow load project as module
sys.path.insert(0, '../../../..')

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [0]:
import matplotlib     
matplotlib.rcParams.update({'font.size': 14})

# Load stellar model grid

In [0]:
from pfsspec.core.grid import ArrayGrid
from pfsspec.stellar.grid import ModelGrid
from pfsspec.stellar.grid.bosz import Bosz
from pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Bosz(pca=True, normalized=True), ArrayGrid)
grid = ModelGrid(Phoenix(pca=True, normalized=True), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

In [0]:
grid.wave

In [0]:
for _, k, axis in grid.enumerate_axes():
    print(k, axis.values)

In [0]:
grid.grid.eigs.keys()

## Count valid spectra

In [0]:
grid.array_grid.value_indexes['flux'].shape, np.sum(grid.array_grid.value_indexes['flux'])

# Plot

In [0]:
M_H = 0
T_eff = 4500
log_g = 1
a_M = 0
C_M = 0

idx = grid.grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
idx

In [0]:
idx = (10, 4, 2, 0, 1)

In [0]:
model = grid.get_model_at(idx)
plt.plot(model.wave, model.flux)

In [0]:
model = grid.get_model_at(idx, denormalize=True)
plt.plot(model.wave, model.flux)

In [0]:
model.get_params()

# Plot eigenvectors

In [0]:
eigv = grid.grid.eigv['flux']
eigv.shape

In [0]:
N = min(5, eigv.shape[-1])

f, axs = plt.subplots(N, 1, figsize=(6, 2 * N), dpi=96)

for i, ax in enumerate(axs):
    ax.plot(grid.wave, eigv[:, i], lw=0.1)
    ax.grid()
    ax.set_ylim(-0.025, 0.025)
    
for i, ax in enumerate(axs[:-1]):
    ax.set_xticklabels([])
    
axs[0].set_ylim(-0.025, 0.005)
    
f.tight_layout()

In [0]:
S = grid.grid.eigs['flux']
S

In [0]:
Y = np.log10(1 - np.cumsum(S) / np.sum(S))
X = np.log10(np.arange(Y.shape[0]) + 1)

f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=120)

ax.plot(X, Y)

ax.set_xlabel(r'$\log_{10} \, k$')
ax.set_ylabel(r'$\log_{10} 1 - \frac{cum sum S_i}{sum S_i}$')

ax.grid()

In [0]:
U = grid.grid.eigv['flux']
U.shape

In [0]:
UU = np.cumsum(U**2, axis=1)
UU.shape

In [0]:
f, ax = plt.subplots(figsize=(7, 4), dpi=120)

for k in [100, 200, 300, 500, 1000]:
    ax.plot(grid.wave, UU[:, k], lw=0.3, label=str(k))
ax.semilogy()
ax.legend()

In [0]:
f, ax = plt.subplots(figsize=(7, 4), dpi=120)

for k, l in zip([100, 200, 300, 500, 1000], [0, 100, 200, 300, 500]):
    ax.plot(grid.wave, UU[:, k] - UU[:, l], lw=0.3, label='{} - {}'.format(k, l))
ax.semilogy()

ax.legend()

# Plot principal components

In [0]:
#s = np.s_[:, :, :, idx[3], idx[4]]
s = np.s_[:, :, :, idx[-1]]

In [0]:
def load_params(name):
    params = grid.array_grid.get_value(name)
    masks = grid.array_grid.value_indexes[name]
        
    return params, masks

In [0]:
def plot_params(params, idx=2, param_idx=0):
    pp = params.shape[idx]
    rr = int(np.ceil(pp / 4 + 0.5))
    f, axs = plt.subplots(rr, 4, figsize=(16, 4 * rr))
    for p in range(pp):
        i = p // 4
        j = p % 4
        
        s = (params.ndim - 1) * [slice(None)]
        s[idx] = p
        s = tuple(s)
        vmin, vmax = params[s][..., param_idx].min(), params[s][..., param_idx].max()
        l = axs[i, j].imshow(params[s][..., param_idx], aspect='auto') #, vmin=vmin, vmax=vmax)
        f.colorbar(l, ax=axs[i, j])
        axs[i, j].set_xlabel('param: {} | slice: {}'.format(param_idx, p))

In [0]:
params, masks = load_params('flux')
params.shape[-1]

In [0]:
for pi in range(min(15, params.shape[-1])):
    plot_params(params[s], param_idx=pi)